#### **Computes the gravitational accelerations on two bodies due to their mutual attraction using Newton’s law of universal gravitation.**
It returns acc1 and acc2 as vectors representing the acceleration of each mass.
This function is essential for integrating the equations of motion.

In [ ]:
def acceleration(r1, r2, m1, m2, G):
    """
    Compute gravitational accelerations on m1 and m2 using Newton's law.
    """
    r = r2 - r1
    dist = np.linalg.norm(r)
    force = G * m1 * m2 * r / dist**3  # Newton's law: F = G * m1 * m2 / r^2 in vector form

    acc1 = +force / m1  # a = F / m
    acc2 = -force / m2  # equal and opposite

    return acc1, acc2

### **Performs a single step of the adaptive Velocity-Verlet integration.**
Uses a time step that adjusts dynamically according to the total magnitude of the acceleration, making it more stable during high-acceleration phases (e.g., near pericenter).
Returns updated positions, velocities, and the adaptive time step.

In [ ]:
def velocity_verlet_adaptive_step(r1, v1, r2, v2, m1, m2, G, tau_0):
    """
    Perform one adaptive step of the Velocity-Verlet integration.
    Returns new r1, v1, r2, v2, and the adaptive time step tau_n.
    """
    # Compute current acceleration
    a1, a2 = acceleration(r1, r2, m1, m2, G)

    # Compute adaptive time step based on acceleration magnitude
    acc_magnitude = np.linalg.norm(a1) + np.linalg.norm(a2)
    tau_n = tau_0 / (1 + acc_magnitude)

    # First half of the Velocity-Verlet step
    r1_new = r1 + v1 * tau_n + 0.5 * a1 * tau_n**2
    r2_new = r2 + v2 * tau_n + 0.5 * a2 * tau_n**2

    # Compute new acceleration
    a1_new, a2_new = acceleration(r1_new, r2_new, m1, m2, G)

    # Second half of the Velocity-Verlet step
    v1_new = v1 + 0.5 * (a1 + a1_new) * tau_n
    v2_new = v2 + 0.5 * (a2 + a2_new) * tau_n

    return r1_new, v1_new, r2_new, v2_new, tau_n

### **Executes one time step of the Symplectic Euler method — a first-order symplectic integrator.**
Updates velocities first using current accelerations, then uses the new velocities to update positions.
It is more stable for long-term orbital simulations than the standard Euler method.

In [ ]:
def symplectic_euler_step(r1, v1, r2, v2, m1, m2, G, dt):
    """
    Perform one step of the Symplectic Euler method for two-body motion.
    Update velocities first, then positions.
    """
    # Compute acceleration at current positions
    a1, a2 = acceleration(r1, r2, m1, m2, G)

    # Update velocities
    v1_new = v1 + dt * a1
    v2_new = v2 + dt * a2

    # Update positions using updated velocities
    r1_new = r1 + dt * v1_new
    r2_new = r2 + dt * v2_new

    return r1_new, v1_new, r2_new, v2_new

### **Initialises the positions and velocities for a two-body system with a small velocity perturbation applied to mass 1.**
Useful for testing sensitivity to initial conditions, such as divergence in chaotic or near-resonant systems.

In [ ]:
def initialize_perturbed_velocity_verlet(a=0.5, delta_v=np.array([0.0, 0.01])):
    """
    Set up initial conditions for a perturbed two-body system.
    delta_v: small perturbation added to velocity of mass 1
    """
    m1 = a
    m2 = 1 - a

    r1 = np.array([1.0, 0.0])
    v1 = np.array([0.0, 2 - np.sqrt(3)]) + delta_v

    r2 = - (m1 / m2) * r1
    v2 = - (m1 / m2) * v1

    return r1, v1, r2, v2, m1, m2

### **Sets up the initial conditions for a symplectic Euler simulation, with a small position perturbation added to body 1.**
Useful for studying stability and divergence in phase space when using a symplectic integrator.

In [ ]:
def initialize_perturbed_symplectic_euler(a=0.5, delta_r=np.array([0.005, 0.0])):
    """
    Initialise perturbed positions for Symplectic Euler method.
    """
    m1 = a
    m2 = 1 - a

    r1 = np.array([1.0, 0.0]) + delta_r
    v1 = np.array([0.0, 2 - np.sqrt(3)])

    r2 = - (m1 / m2) * r1
    v2 = - (m1 / m2) * v1

    return r1, v1, r2, v2, m1, m2

### **Implements the anisotropic 2D FTCS (Forward-Time Central-Space) scheme for the heat equation.**
Supports different thermal conductivities in the x and y directions ($\kappa_x$, $\kappa_y$).
Used to simulate anisotropic heat flow where diffusion is direction-dependent.

In [ ]:
def ftcs_anisotropic(T, kappa_x, kappa_y, tau, h):
    """
    Perform one time step of the anisotropic FTCS scheme for 2D diffusion.
    """
    Nx, Ny = T.shape
    T_new = T.copy()
    
    for i in range(1, Nx - 1):
        for j in range(1, Ny - 1):
            T_new[i, j] = T[i, j] + tau * (
                kappa_x * (T[i+1, j] + T[i-1, j] - 2*T[i, j]) / h**2 +
                kappa_y * (T[i, j+1] + T[i, j-1] - 2*T[i, j]) / h**2
            )

    return T_new

### **Heatmap Plot of Temperature Field**

In [ ]:
plt.figure(figsize=(6, 5))
plt.imshow(T, cmap='hot', origin='lower')
plt.colorbar(label='Temperature')
plt.title(f'Anisotropic Heat Diffusion after {n_steps} steps')
plt.xlabel('x')
plt.ylabel('y')
plt.show()